# Models baseline

# 1. Imports

## 1.1 Packages

In [9]:
import numpy as np
import pandas as pd

from sklearn.ensemble import HistGradientBoostingRegressor

from store_sales.pipelines.model_training.nodes import create_lag_features, create_rolling_mean_features

## 1.2 Options

In [10]:
lags = [20, 30, 45, 60, 90, 120, 365, 730]
alphas = [0.95, 0.9, 0.8, 0.7, 0.5]

## 1.3 Dataset

In [11]:
df_training = pd.read_parquet("../data/04_feature/df_train_feature_engineered.parquet")

# df_test = pd.read_parquet("../data/04_feature/df_test_feature_engineered.parquet")

# 2. Hist Gradient Boosting model

In [12]:
df_training.sample(2)

,id,date,store_nbr,family,sales,onpromotion,transactions,city,state,type,...,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,season,holiday_type,is_workday,is_train,is_validation
1546572,1526970,2015-05-09,52,PLAYERS AND ELECTRONICS,0.0,0,NaN,Manta,Manabi,A,...,False,False,False,False,False,1,Additional_National,1,1,0
299537,297755,2013-06-17,13,PREPARED FOODS,12.0,0,914.0,Latacunga,Cotopaxi,C,...,False,False,False,False,False,2,None,1,1,0


In [13]:
df_train = df_training[df_training["is_train"] == 1]
df_valid = df_training[df_training["is_validation"] == 1]

In [14]:
# Create lag features
df_train = create_lag_features(df_train, lags)
df_train = create_rolling_mean_features(df_train, lags)

df_valid = create_lag_features(df_valid, lags)
df_valid = create_rolling_mean_features(df_valid, lags)

In [15]:
df_train.tail(5)

,id,date,store_nbr,family,sales,onpromotion,transactions,city,state,type,...,sales_lag_365,sales_lag_730,rolling_mean_20,rolling_mean_30,rolling_mean_45,rolling_mean_60,rolling_mean_90,rolling_mean_120,rolling_mean_365,rolling_mean_730
3020357,2966897,2017-07-27,54,SEAFOOD,2.0,0,662.0,El Carmen,Manabi,C,...,0.0,3.0,3.45,2.700000,2.911111,2.866667,2.911111,2.700000,1.958904,1.757534
3022139,2968679,2017-07-28,54,SEAFOOD,4.0,0,766.0,El Carmen,Manabi,C,...,2.0,0.0,3.45,2.733333,2.955556,2.783333,2.933333,2.700000,1.964384,1.756164
3023921,2970461,2017-07-29,54,SEAFOOD,4.0,2,870.0,El Carmen,Manabi,C,...,1.0,0.0,3.20,2.833333,3.000000,2.850000,2.955556,2.716667,1.969863,1.761644
3025703,2972243,2017-07-30,54,SEAFOOD,4.0,0,1108.0,El Carmen,Manabi,C,...,4.0,1.0,3.10,2.966667,3.022222,2.916667,2.988889,2.750000,1.978082,1.767123
3027485,2974025,2017-07-31,54,SEAFOOD,4.0,0,860.0,El Carmen,Manabi,C,...,2.0,1.0,2.80,3.033333,3.044444,2.933333,3.011111,2.775000,1.978082,1.771233


In [16]:
y_train = df_train["sales"]
x_train = df_train.drop(columns=["id", "date"])

y_valid = df_valid["sales"]
x_valid = df_valid.drop(columns=["id", "date"])

In [ ]:
# Train the model
model = HistGradientBoostingRegressor().fit(x_train, y_train)

# Predict on the train and the validation set
y_pred_train = model.predict(x_train)
y_pred_valid = model.predict(x_valid)